<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item MAE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [2]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit.metrics.predict import mae
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [3]:
def model_trainer(neighbours):
    neighbours=int(neighbours)
    print(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,aggregate='weighted-average') # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = mae(all_recs['prediction'], all_recs['rating'])
    return metric

In [6]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': model_trainer(x), 'status': STATUS_OK }

average=[[] for _ in range(10)]

for y in range(len(average)):
    best = fmin(objective,
        space=hp.uniform('x', 1, 50),
        algo=tpe.suggest,
        max_evals=10
               )
    average[y].append(best)

48                                                                                                                     
16                                                                                                                     
15                                                                                                                     
44                                                                                                                     
3                                                                                                                      
23                                                                                                                     
40                                                                                                                     
3                                                                                                                      
40                                      

In [8]:
average = [[{'x': 16.868565591621376}],
 [{'x': 18.27334308452329}],
 [{'x': 13.807761918320374}],
 [{'x': 32.3520268685702}],
 [{'x': 17.923088369224278}],
 [{'x': 19.130390107556554}],
 [{'x': 37.58890812465468}],
 [{'x': 15.478081716299211}],
 [{'x': 29.42799196079218}],
 [{'x': 43.86544666543126}]]

In [9]:
flat_list = [item for sublist in average for item in sublist]

In [10]:
mae = [0.7042934576506902,0.7067916436578668,0.7064144666189,0.7003264536819339,0.7035099920792894,0.7001105328078204,
        0.7049136415550669,0.7031425034780152,0.6984885984667524,0.7065210179202679]
mae = pd.DataFrame(mae)

In [12]:
average = pd.DataFrame.from_dict(flat_list)
metric = pd.concat([average,mae],axis=1)
metric.columns = ['Neighbours','MAE']
metric

,Neighbours,MAE
0,16.868566,0.704293
1,18.273343,0.706792
2,13.807762,0.706414
3,32.352027,0.700326
4,17.923088,0.703510
5,19.130390,0.700111
6,37.588908,0.704914
7,15.478082,0.703143
8,29.427992,0.698489
9,43.865447,0.706521


In [13]:
metric.mean()

Neighbours    24.471560
MAE            0.703451
dtype: float64